In [1]:
import requests
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from unidecode import unidecode
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.84 Safari/537.36'}




In [14]:
lgs = {
	'ENG1':47,
	'ENG2':48,
	'ENG3':108,
	'ENG4':109,
	'GER1':54,
	'GER2':146,
	'FRA1':53,
	'FRA2':110,
	'SPA1':87,
	'SPA2':140,
	'POR1':61,
	'ITA1':55,
	'ITA2':86
}
rows = []
for lg in lgs:
	for year in range(2020, 2022):
		print(year, lg)
		schedule = requests.get(f'https://www.fotmob.com/api/fixtures?id={lgs[lg]}&season={year}%2F{year+1}', headers = headers)
		while schedule.status_code != 200:
			schedule = requests.get(f'https://www.fotmob.com/api/fixtures?id={lgs[lg]}&season={year}%2F{year+1}', headers = headers)

		matches = schedule.json()
		for match in matches:
			if match['status']['finished'] == True and match['status']['cancelled'] == False:
				time.sleep(random.randint(1, 12))
				sleep = random.randint(0, 100)
				if sleep > 95:
					time.sleep(random.randint(150, 240))
				
				odds_response = requests.get(f'https://www.fotmob.com/api/matchOdds?matchId={match["id"]}&ccode3=USA_CO&bettingProvider=Bet365_Colorado', headers = headers)
				while odds_response.status_code != 200:
					odds_response = requests.get(f'https://www.fotmob.com/api/matchOdds?matchId={match["id"]}&ccode3=USA_CO&bettingProvider=Bet365_Colorado', headers = headers)
				if len(odds_response.json()) > 2 and 'whoWins' in odds_response.json()['bets'].keys():
					team1odds = odds_response.json()['bets']['whoWins']['coeffs'][0][1]
					drawodds = odds_response.json()['bets']['whoWins']['coeffs'][1][1]
					team2odds = odds_response.json()['bets']['whoWins']['coeffs'][2][1]
				else:
					team1odds = -1
					drawodds = -1
					team2odds = -1			
				
				match_response = requests.get('https://www.fotmob.com/api/matchDetails?matchId=' + match['id'], headers = headers)
				while match_response.status_code != 200:
					match_response = requests.get('https://www.fotmob.com/api/matchDetails?matchId=' + match['id'], headers = headers)

				teams = match_response.json()['content']['lineup']['lineup']
				date = match_response.json()['general']['matchTimeUTC']
				team1 = teams[0]['teamName']
				team2 = teams[1]['teamName']
				score = match_response.json()['header']['status']['scoreStr']
				i = 0
				name = teams[0]['teamName']
				for pos_idx in teams[0]['players']:
					for player in pos_idx:
						row = {}
						row['year'] = year+1
						row['comp'] = lg
						row['playerid'] = player['id']
						row['date'] = date
						row['team'] = team1
						row['opponent'] = team2
						row['team_score'] = score.split(' - ')[0]
						row['opponent_score'] = score.split(' - ')[1]
						row['team_odds'] = team1odds
						row['draw_odds'] = drawodds
						row['opponent_odds'] = team2odds
						row['name'] = player['name']['fullName']
						row['pos'] = player['positionStringShort']
						row['pos_group'] = i

						row['usual_pos'] = player['usualPosition']
						row['shirt'] = player['shirt']
						if 'stats' in player.keys():
							for stat_idx in range(0, len(player['stats'])):
								for stat in player['stats'][stat_idx]['stats']:
									row[player['stats'][stat_idx]['stats'][stat]['key']] = player['stats'][stat_idx]['stats'][stat]['value']
							rows.append(row)
					i += 1
				for player in teams[0]['bench']:
					row = {}
					row['year'] = year+1
					row['comp'] = lg
					row['playerid'] = player['id']
					row['date'] = date
					row['team'] = team1
					row['opponent'] = team2
					row['team_score'] = score.split(' - ')[0]
					row['opponent_score'] = score.split(' - ')[1]
					row['team_odds'] = team1odds
					row['draw_odds'] = drawodds
					row['opponent_odds'] = team2odds
					row['name'] = player['name']['fullName']
					row['pos'] = player['position']
					row['usual_pos'] = player['usualPosition']
					row['shirt'] = player['shirt']
					if 'stats' in player.keys():
						for stat_idx in range(0, len(player['stats'])):
							for stat in player['stats'][stat_idx]['stats']:
								row[player['stats'][stat_idx]['stats'][stat]['key']] = player['stats'][stat_idx]['stats'][stat]['value']
						rows.append(row)


				name = teams[1]['teamName']
				i = 0
				for pos_idx in teams[1]['players']:
					for player in pos_idx:
						row = {}
						row['year'] = year+1
						row['comp'] = lg
						row['playerid'] = player['id']
						row['date'] = date
						row['team'] = team2
						row['opponent'] = team1
						row['team_score'] = score.split(' - ')[1]
						row['opponent_score'] = score.split(' - ')[0]
						row['team_odds'] = team2odds
						row['draw_odds'] = drawodds
						row['opponent_odds'] = team1odds
						row['name'] = player['name']['fullName']
						row['pos'] = player['positionStringShort']
						row['pos_group'] = i
						row['usual_pos'] = player['usualPosition']
						row['shirt'] = player['shirt']
						if 'stats' in player.keys():
							for stat_idx in range(0, len(player['stats'])):
								for stat in player['stats'][stat_idx]['stats']:
									row[player['stats'][stat_idx]['stats'][stat]['key']] = player['stats'][stat_idx]['stats'][stat]['value']
							rows.append(row)
					i += 1
				for player in teams[1]['bench']:
					row = {}
					row['year'] = year+1
					row['comp'] = lg
					row['playerid'] = player['id']
					row['date'] = date
					row['team'] = team2
					row['opponent'] = team1
					row['team_score'] = score.split(' - ')[1]
					row['opponent_score'] = score.split(' - ')[0]
					row['team_odds'] = team2odds
					row['draw_odds'] = drawodds
					row['opponent_odds'] = team1odds
					row['name'] = player['name']['fullName']
					row['pos'] = player['position']
					row['usual_pos'] = player['usualPosition']
					row['shirt'] = player['shirt']
					if 'stats' in player.keys():
						for stat_idx in range(0, len(player['stats'])):
							for stat in player['stats'][stat_idx]['stats']:
								row[player['stats'][stat_idx]['stats'][stat]['key']] = player['stats'][stat_idx]['stats'][stat]['value']
						rows.append(row)

				
rated_df = pd.DataFrame(rows)
rated_df['date'] = rated_df['date'].astype('str')
rated_df['date'] = pd.to_datetime(rated_df['date'], format='mixed').dt.date
rated_df['name'] = rated_df['name'].apply(unidecode)
rated_df.to_csv('game_ratings.csv')

2020 ENG1
2021 ENG1
2020 ENG2
2021 ENG2
2020 ENG3


KeyboardInterrupt: 

In [34]:

url = "https://fbref.com/en/squads/822bd0ba/2022-2023/matchlogs/c9/schedule/Liverpool-Scores-and-Fixtures-Premier-League"
headers = {'User-Agent': 'Mozilla/5.0'}
r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
hrefs = soup.find_all('a')
links = []
for link in hrefs:
	if link.text == "Match Report":
		links.append(link)

In [131]:

import random
dfs = []
for link in links:
	r = requests.get('https://fbref.com/' + link['href'], headers=headers)
	soup = BeautifulSoup(r.text, 'html.parser')
	date = soup.find_all('h1')[0].text.split('Match Report – ')[1]
	team1 = soup.find_all('h2')[3].text.replace(" Player Stats", "")
	team2 = soup.find_all('h2')[10].text.replace(" Player Stats", "")
	team1stats = pd.read_html('https://fbref.com/' + link['href'])[3]
	team1stats['Team'] = team1
	team1stats['Opponent'] = team2
	team2stats = pd.read_html('https://fbref.com/' + link['href'])[10]
	team2stats['Team'] = team2
	team2stats['Opponent'] = team1
	game_stats = pd.concat([team1stats, team2stats])
	game_stats.columns = ['Player', '#',  'Nation','Pos','Age','Min','Gls','Ast','PK','PKatt','Sh','SoT','CrdY','CrdR','Touches','Tkl','Int','Blocks','xG','npxG','xAG','SCA','GCA','pass_cmp','pass_att','pass_cmppct','prog_pass','carries','prog_carries','takeon_att','takeon_succ', 'team', 'opponent']
	game_stats = game_stats[~game_stats['Player'].str.contains('Players')]
	game_stats['date'] = date
	game_stats = game_stats[['Player', '#', 'Nation','Pos','Age','team', 'opponent', 'date', 'Min','Gls','Ast','PK','PKatt','Sh','SoT','CrdY','CrdR','Touches','Tkl','Int','Blocks','xG','npxG','xAG','SCA','GCA','pass_cmp','pass_att','pass_cmppct','prog_pass','carries','prog_carries','takeon_att','takeon_succ']]
	time.sleep(random.randint(12, 30))
	dfs.append(game_stats)
microstats = pd.concat(dfs).sort_values('date')
microstats['date'] = microstats['date'].str.replace('North West Derby', '').str.replace('Merseyside Derby', '')
microstats['date'] = pd.to_datetime(microstats['date'])
microstats['Player'] = microstats['Player'].apply(unidecode)
microstats.to_csv('microstats.csv')

In [276]:
train_df = pd.read_csv('pl_ratings_202122.csv')
train_df.columns

Index(['Unnamed: 0', 'playerid', 'date', 'team', 'opponent', 'team_score',
       'opponent_score', 'team_odds', 'draw_odds', 'opponent_odds', 'name',
       'pos', 'pos_group', 'usual_pos', 'shirt', 'rating_title',
       'minutes_played', 'saves', 'goals_conceded',
       'expected_goals_on_target_faced', 'goals_prevented', 'accurate_passes',
       'long_balls_accurate', 'keeper_diving_save', 'saves_inside_box',
       'keeper_sweeper', 'punches', 'player_throws', 'keeper_high_claim',
       'recoveries', 'expected_assists', 'fantasy_points', 'touches', 'goals',
       'assists', 'total_shots', 'chances_created', 'xg_and_xa',
       'dribbles_succeeded', 'passes_into_final_third', 'dispossessed',
       'tackles_succeeded', 'clearances', 'defensive_actions', 'duel_won',
       'duel_lost', 'ground_duels_won', 'aerials_won', 'was_fouled', 'fouls',
       'headed_clearance', 'interceptions', 'shot_blocks', 'dribbled_past',
       'Unnamed: 54', 'expected_goals', 'expected_goals_on_tar

In [329]:
train_df = pd.concat([pd.read_csv('pl_ratings_202122.csv'), pd.read_csv('pl_ratings_202223.csv'),])
gk_cols = ['rating_title','minutes_played','goals_conceded',
	'expected_goals_on_target_faced','goals_prevented',
	'keeper_diving_save','saves_inside_box',
	'keeper_sweeper','punches','player_throws',
	'keeper_high_claim','recoveries',
	'touches','goals','assists','total_shots','chances_created',
	'expected_assists','save_pct','save_cmp','pass_pct','pass_cmp',
	'long_ball_pct','long_ball_cmp']

fwd_cols = ['rating_title','minutes_played','recoveries',
	'goals','assists','total_shots','chances_created','expected_assists',
	'xg_and_xa','dispossessed','clearances',
	'interceptions','defensive_actions','duel_won','duel_lost',
	'fouls','headed_clearance','owngoal','shot_blocks',
	'expected_goals','blocked_shots','expected_goals_non_penalty',
	'expected_goals_on_target_variant','big_chance_missed_title',
	'corners','clearance_off_the_line','penalties_won',
	'conceded_penalties','missed_penalty','error_led_to_goal',
	'pass_cmp','dribbles_cmp','crosses_cmp',
	'shots_ot']
mid_cols = ['rating_title','minutes_played','recoveries',
	'goals','assists','total_shots','chances_created','expected_assists',
	'xg_and_xa','dispossessed','clearances',
	'interceptions','defensive_actions','duel_won','duel_lost','was_fouled',
	'dribbled_past','owngoal','shot_blocks',
	'expected_goals','blocked_shots','expected_goals_non_penalty',
	'expected_goals_on_target_variant','big_chance_missed_title',
	'corners','Offsides','clearance_off_the_line','penalties_won',
	'conceded_penalties','last_man_tackle','missed_penalty','error_led_to_goal',
	'dribbles_cmp',
	'tackles_cmp','duels_cmp',
	'shots_ot']
def_cols = ['rating_title','minutes_played','recoveries',
	'goals','assists','total_shots','chances_created','expected_assists',
	'xg_and_xa','passes_into_final_third','dispossessed','clearances',
	'interceptions','defensive_actions','duel_won','duel_lost','was_fouled',
	'fouls','headed_clearance','dribbled_past','owngoal','shot_blocks',
	'expected_goals','expected_goals_non_penalty',
	'expected_goals_on_target_variant','big_chance_missed_title',
	'corners','Offsides','clearance_off_the_line','penalties_won',
	'conceded_penalties','last_man_tackle','error_led_to_goal',
	'long_ball_cmp','dribbles_cmp','shots_ot']
train_df[['save','save_pct']] = train_df['saves'].str.split(' \(',expand=True)
train_df['save_pct'] = train_df['save_pct'].str.replace('%)', '')
train_df[['save_att','save_cmp']] = train_df['save'].str.split('/',expand=True)
train_df = train_df.drop(['save','saves'], axis=1)
train_df[['pass','pass_pct']] = train_df['accurate_passes'].str.split(' \(',expand=True)
train_df['pass_pct'] = train_df['pass_pct'].str.replace('%)', '')
train_df[['pass_att','pass_cmp']] = train_df['pass'].str.split('/',expand=True)
train_df = train_df.drop(['pass', 'accurate_passes'], axis=1)
train_df[['long_ball','long_ball_pct']] = train_df['long_balls_accurate'].str.split(' \(',expand=True)
train_df['long_ball_pct'] = train_df['long_ball_pct'].str.replace('%)', '')
train_df[['long_ball_att','long_ball_cmp']] = train_df['long_ball'].str.split('/',expand=True)
train_df = train_df.drop(['long_ball','long_balls_accurate'], axis=1)
train_df[['dribbles','dribble_pct']] = train_df['dribbles_succeeded'].str.split(' \(',expand=True)
train_df['dribble_pct'] = train_df['dribble_pct'].str.replace('%)', '')
train_df[['dribbles_att','dribbles_cmp']] = train_df['dribbles'].str.split('/',expand=True)
train_df = train_df.drop(['dribbles', 'dribbles_succeeded'], axis=1)
train_df[['crosses','cross_pct']] = train_df['accurate_crosses'].str.split(' \(',expand=True)
train_df['cross_pct'] = train_df['cross_pct'].str.replace('%)', '')
train_df[['crosses_att','crosses_cmp']] = train_df['crosses'].str.split('/',expand=True)
train_df = train_df.drop(['crosses', 'accurate_crosses'], axis=1)
train_df[['tackles','tackles_pct']] = train_df['tackles_succeeded'].str.split(' \(',expand=True)
train_df['tackles_pct'] = train_df['tackles_pct'].str.replace('%)', '')
train_df[['tackles_att','tackles_cmp']] = train_df['tackles'].str.split('/',expand=True)
train_df = train_df.drop(['tackles', 'tackles_succeeded'], axis=1)
train_df[['duels','duel_pct']] = train_df['ground_duels_won'].str.split(' \(',expand=True)
train_df['duel_pct'] = train_df['duel_pct'].str.replace('%)', '')
train_df[['duels_att','duels_cmp']] = train_df['duels'].str.split('/',expand=True)
train_df = train_df.drop(['duels', 'ground_duels_won'], axis=1)
train_df[['aerials','aerial_pct']] = train_df['aerials_won'].str.split(' \(',expand=True)
train_df['aerial_pct'] = train_df['aerial_pct'].str.replace('%)', '')
train_df[['aerial_att','aerial_cmp']] = train_df['aerials'].str.split('/',expand=True)
train_df = train_df.drop(['aerials', 'aerials_won'], axis=1)
train_df[['sot','sot_pct']] = train_df['shot_accuracy'].str.split(' \(',expand=True)
train_df['sot_pct'] = train_df['sot_pct'].str.replace('%)', '')
train_df[['shots_att','shots_ot']] = train_df['sot'].str.split('/',expand=True)
train_df = train_df.drop(['sot', 'shot_accuracy'], axis=1)
train_gk = train_df[(train_df['usual_pos'] == 0) & (train_df['minutes_played'] > 45)][gk_cols].fillna(value=0)
train_def = train_df[(train_df['usual_pos'] == 1) & (train_df['minutes_played'] > 45)][def_cols].fillna(value=0)
train_mid = train_df[(train_df['usual_pos'] == 2) & (train_df['minutes_played'] > 45)][mid_cols].fillna(value=0)
train_fwd = train_df[(train_df['usual_pos'] == 3) & (train_df['minutes_played'] > 45)][fwd_cols].fillna(value=0)
train_df.to_csv('cleantrain.csv')

In [330]:
test_df = pd.read_csv('pl_ratings_202324.csv')

test_df[['save','save_pct']] = test_df['saves'].str.split(' \(',expand=True)
test_df['save_pct'] = test_df['save_pct'].str.replace('%)', '')
test_df[['save_att','save_cmp']] = test_df['save'].str.split('/',expand=True)
test_df = test_df.drop(['save','saves'], axis=1)
test_df[['pass','pass_pct']] = test_df['accurate_passes'].str.split(' \(',expand=True)
test_df['pass_pct'] = test_df['pass_pct'].str.replace('%)', '')
test_df[['pass_att','pass_cmp']] = test_df['pass'].str.split('/',expand=True)
test_df = test_df.drop(['pass', 'accurate_passes'], axis=1)
test_df[['long_ball','long_ball_pct']] = test_df['long_balls_accurate'].str.split(' \(',expand=True)
test_df['long_ball_pct'] = test_df['long_ball_pct'].str.replace('%)', '')
test_df[['long_ball_att','long_ball_cmp']] = test_df['long_ball'].str.split('/',expand=True)
test_df = test_df.drop(['long_ball','long_balls_accurate'], axis=1)
test_df[['dribbles','dribble_pct']] = test_df['dribbles_succeeded'].str.split(' \(',expand=True)
test_df['dribble_pct'] = test_df['dribble_pct'].str.replace('%)', '')
test_df[['dribbles_att','dribbles_cmp']] = test_df['dribbles'].str.split('/',expand=True)
test_df = test_df.drop(['dribbles', 'dribbles_succeeded'], axis=1)
test_df[['crosses','cross_pct']] = test_df['accurate_crosses'].str.split(' \(',expand=True)
test_df['cross_pct'] = test_df['cross_pct'].str.replace('%)', '')
test_df[['crosses_att','crosses_cmp']] = test_df['crosses'].str.split('/',expand=True)
test_df = test_df.drop(['crosses', 'accurate_crosses'], axis=1)
test_df[['tackles','tackles_pct']] = test_df['tackles_succeeded'].str.split(' \(',expand=True)
test_df['tackles_pct'] = test_df['tackles_pct'].str.replace('%)', '')
test_df[['tackles_att','tackles_cmp']] = test_df['tackles'].str.split('/',expand=True)
test_df = test_df.drop(['tackles', 'tackles_succeeded'], axis=1)
test_df[['duels','duel_pct']] = test_df['ground_duels_won'].str.split(' \(',expand=True)
test_df['duel_pct'] = test_df['duel_pct'].str.replace('%)', '')
test_df[['duels_att','duels_cmp']] = test_df['duels'].str.split('/',expand=True)
test_df = test_df.drop(['duels', 'ground_duels_won'], axis=1)
test_df[['aerials','aerial_pct']] = test_df['aerials_won'].str.split(' \(',expand=True)
test_df['aerial_pct'] = test_df['aerial_pct'].str.replace('%)', '')
test_df[['aerial_att','aerial_cmp']] = test_df['aerials'].str.split('/',expand=True)
test_df = test_df.drop(['aerials', 'aerials_won'], axis=1)
test_df[['sot','sot_pct']] = test_df['shot_accuracy'].str.split(' \(',expand=True)
test_df['sot_pct'] = test_df['sot_pct'].str.replace('%)', '')
test_df[['shots_att','shots_ot']] = test_df['sot'].str.split('/',expand=True)
test_df = test_df.drop(['sot', 'shot_accuracy'], axis=1)
test_gk = test_df[(test_df['usual_pos'] == 0) & (test_df['minutes_played'] > 45)][gk_cols].fillna(value=0)
test_def = test_df[(test_df['usual_pos'] == 1) & (test_df['minutes_played'] > 45)][def_cols].fillna(value=0)
test_mid = test_df[(test_df['usual_pos'] == 2) & (test_df['minutes_played'] > 45)][mid_cols].fillna(value=0)
test_fwd = test_df[(test_df['usual_pos'] == 3) & (test_df['minutes_played'] > 45)][fwd_cols].fillna(value=0)
test_df.to_csv('cleantest.csv')

In [331]:
from sklearn.linear_model import Ridge

data = train_gk.values
X, y = data[:, 2:], data[:, 0]
# define model
gk_model = Ridge(alpha=1.0)
# fit model
gk_model.fit(X, y)

data = train_def.values
X, y = data[:, 2:], data[:, 0]
# define model
def_model = Ridge(alpha=1.0)
# fit model
def_model.fit(X, y)

data = train_mid.values
X, y = data[:, 2:], data[:, 0]
# define model
mid_model = Ridge(alpha=1.0)
# fit model
mid_model.fit(X, y)

data = train_fwd.values
X, y = data[:, 2:], data[:, 0]
# define model
fwd_model = Ridge(alpha=1.0)
# fit model
fwd_model.fit(X, y)

Ridge()

In [332]:
coef = pd.DataFrame()
coef['variable'] = def_cols[2:]
coef['def_coef'] = def_model.coef_
coef

,variable,def_coef
0,recoveries,0.056260
1,goals,0.926338
2,assists,0.648483
3,total_shots,0.037824
4,chances_created,0.159896
5,expected_assists,0.101513
6,xg_and_xa,0.054619
7,passes_into_final_third,0.027575
8,dispossessed,-0.079397
9,clearances,0.009573


In [402]:
# define new data

# make a prediction
clean_gk = test_df[test_df['usual_pos'] == 0][test_df.minutes_played > 45].fillna(value=0)
clean_gk['predicted_rating'] = gk_model.predict(clean_gk[gk_cols].values[:, 2:])
clean_def = test_df[test_df['usual_pos'] == 1][test_df.minutes_played > 45].fillna(value=0)
clean_def['predicted_rating'] = def_model.predict(clean_def[def_cols].values[:, 2:])
clean_mid = test_df[test_df['usual_pos'] == 2][test_df.minutes_played > 45].fillna(value=0)
clean_mid['predicted_rating'] = mid_model.predict(clean_mid[mid_cols].values[:, 2:])
clean_fwd = test_df[test_df['usual_pos'] == 3][test_df.minutes_played > 45].fillna(value=0)
clean_fwd['predicted_rating'] = fwd_model.predict(clean_fwd[fwd_cols].values[:, 2:])
final_test = pd.concat([clean_gk, clean_def, clean_mid, clean_fwd])
final_test['predicted_coef'] = ((final_test['predicted_rating']-8.5)**2)*(-1/50)+1
final_test['predicted_rating'] = final_test['predicted_rating']*final_test['predicted_coef']



C:\Users\Jetson.Ku\AppData\Local\Temp\ipykernel_30272\4166633396.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_gk = test_df[test_df['usual_pos'] == 0][test_df.minutes_played > 45].fillna(value=0)
C:\Users\Jetson.Ku\AppData\Local\Temp\ipykernel_30272\4166633396.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_def = test_df[test_df['usual_pos'] == 1][test_df.minutes_played > 45].fillna(value=0)
C:\Users\Jetson.Ku\AppData\Local\Temp\ipykernel_30272\4166633396.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_mid = test_df[test_df['usual_pos'] == 2][test_df.minutes_played > 45].fillna(value=0)
C:\Users\Jetson.Ku\AppData\Local\Temp\ipykernel_30272\4166633396.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  clean_fwd = test_df[test_df['usual_pos'] == 3][test_df.minutes_played > 45].fillna(value=0)


In [353]:
final_test['residual'] = final_test['predicted_rating'] - final_test['rating_title']
final_test.sort_values(by='predicted_rating')[['date', 'team', 'opponent', 'name', 'rating_title', 'predicted_rating']]

,date,team,opponent,name,rating_title,predicted_rating
1375,2023-09-02,Fulham,Manchester City,Bernd Leno,3.95,3.454147
2367,2023-09-30,Brighton & Hove Albion,Aston Villa,Jason Steele,4.36,3.540767
2298,2023-09-24,Sheffield United,Newcastle United,Wesley Foderingham,4.00,3.717196
2814,2023-10-07,Burnley,Chelsea,James Trafford,5.45,4.554706
1266,2023-09-02,Burnley,Tottenham Hotspur,Ameen Al Dakhil,3.21,4.643928
...,...,...,...,...,...,...
538,2023-08-19,Brighton & Hove Albion,Wolverhampton Wanderers,Solly March,9.37,8.295957
1395,2023-09-02,Brighton & Hove Albion,Newcastle United,Evan Ferguson,9.70,8.303982
2321,2023-09-24,Newcastle United,Sheffield United,Bruno Guimaraes,9.21,8.322308
1355,2023-09-02,Manchester City,Fulham,Erling Haaland,9.74,8.421051


In [407]:
final_test.sort_values(by='predicted_rating')

,Unnamed: 0,playerid,date,team,opponent,team_score,opponent_score,team_odds,draw_odds,opponent_odds,...,duels_att,duels_cmp,aerial_pct,aerial_att,aerial_cmp,sot_pct,shots_att,shots_ot,predicted_rating,predicted_coef
1375,1375,215168,2023-09-02,Fulham,Manchester City,1,5,13.00,8.00,1.18,...,0,0,0,0,0,0,0,0,1.724320,0.495701
2367,2367,107917,2023-09-30,Brighton & Hove Albion,Aston Villa,1,6,2.00,3.00,2.00,...,0,0,0,0,0,0,0,0,1.831917,0.513462
2298,2298,212944,2023-09-24,Sheffield United,Newcastle United,0,8,6.00,4.33,1.53,...,0,0,0,0,0,0,0,0,2.058015,0.548780
2814,2814,1187213,2023-10-07,Burnley,Chelsea,1,4,5.25,4.00,1.67,...,0,0,0,0,0,0,0,0,3.245106,0.700677
1266,1266,1279764,2023-09-02,Burnley,Tottenham Hotspur,2,5,3.50,3.80,1.95,...,2,9,0,0,3,0,0,1,3.381390,0.715312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2347,2347,540088,2023-09-30,Aston Villa,Brighton & Hove Albion,6,1,2.00,3.00,2.00,...,3,5,25,1,4,80,4,5,9.501969,0.836500
538,538,416850,2023-08-19,Brighton & Hove Albion,Wolverhampton Wanderers,4,1,1.80,4.00,4.20,...,7,12,0,0,2,100,4,4,9.508400,0.961281
1395,1395,1068482,2023-09-02,Brighton & Hove Albion,Newcastle United,3,1,2.50,3.80,2.50,...,6,8,100,1,1,100,4,4,9.517618,0.960004
2321,2321,850354,2023-09-24,Newcastle United,Sheffield United,8,0,1.53,4.33,6.00,...,6,6,100,2,2,100,1,1,9.538093,0.956944


In [413]:
season = final_test[['playerid', 'minutes_played', 'rating_title', 'predicted_rating']].groupby('playerid').mean()
season['diff'] = season['predicted_rating'] - season['rating_title']
season.sort_values(by="predicted_rating", ascending=False)

,minutes_played,rating_title,predicted_rating,diff
playerid,,,,
521148,90.000000,9.040000,9.094026,0.054026
1073742,67.000000,8.840000,9.032627,0.192627
751649,78.000000,8.890000,9.015466,0.125466
318615,66.000000,8.400000,8.510302,0.110302
822237,74.000000,8.340000,8.147184,-0.192816
...,...,...,...,...
1089685,81.000000,6.300000,5.394956,-0.905044
905498,75.500000,5.720000,5.376487,-0.343513
107917,90.000000,6.136000,5.193636,-0.942364


In [314]:
import pickle

with open('fwd_fotmob_rating_model.pkl', 'wb') as f:
  pickle.dump(fwd_model, f)
with open('mid_fotmob_rating_model.pkl', 'wb') as f:
  pickle.dump(mid_model, f)
with open('def_fotmob_rating_model.pkl', 'wb') as f:
  pickle.dump(def_model, f)